In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model
import statsmodels.formula.api as smf
from sqlalchemy import create_engine
%matplotlib inline

import warnings
warnings.filterwarnings(action="ignore")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'


engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weather_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

engine.dispose()

weather_df.head(10)

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
5,2006-04-01 03:00:00+00:00,Partly Cloudy,rain,9.222222,7.111111,0.85,13.9587,258.0,14.9569,0.0,1016.66,Partly cloudy throughout the day.
6,2006-04-01 04:00:00+00:00,Partly Cloudy,rain,7.733333,5.522222,0.95,12.3648,259.0,9.9820,0.0,1016.72,Partly cloudy throughout the day.
7,2006-04-01 05:00:00+00:00,Partly Cloudy,rain,8.772222,6.527778,0.89,14.1519,260.0,9.9820,0.0,1016.84,Partly cloudy throughout the day.
8,2006-04-01 06:00:00+00:00,Partly Cloudy,rain,10.822222,10.822222,0.82,11.3183,259.0,9.9820,0.0,1017.37,Partly cloudy throughout the day.
9,2006-04-01 07:00:00+00:00,Partly Cloudy,rain,13.772222,13.772222,0.72,12.5258,279.0,9.9820,0.0,1017.22,Partly cloudy throughout the day.


In [5]:
weather_df['temp diff']=weather_df['apparenttemperature'] - weather_df['temperature']

In [6]:
weather_df.isnull().mean()

date                   0.0
summary                0.0
preciptype             0.0
temperature            0.0
apparenttemperature    0.0
humidity               0.0
windspeed              0.0
windbearing            0.0
visibility             0.0
loudcover              0.0
pressure               0.0
dailysummary           0.0
temp diff              0.0
dtype: float64

In [8]:
weather_df.describe()

,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,temp diff
count,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.0,96453.000000,96453.000000
mean,11.932678,10.855029,0.734899,10.810640,187.509232,10.347325,0.0,1003.235956,-1.077650
std,9.551546,10.696847,0.195473,6.913571,107.383428,4.192123,0.0,116.969906,1.678694
min,-21.822222,-27.716667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-10.183333
25%,4.688889,2.311111,0.600000,5.828200,116.000000,8.339800,0.0,1011.900000,-2.216667
50%,12.000000,12.000000,0.780000,9.965900,180.000000,10.046400,0.0,1016.450000,0.000000
75%,18.838889,18.838889,0.890000,14.135800,290.000000,14.812000,0.0,1021.090000,0.000000
max,39.905556,39.344444,1.000000,63.852600,359.000000,16.100000,0.0,1046.380000,4.811111


In [10]:
X= weather_df[['humidity','windspeed']]
Y= weather_df['temp diff']

import statsmodels.api as sm

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              temp diff   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Sun, 18 Aug 2019   Prob (F-statistic):               0.00
Time:                        16:02:17   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.0

In [11]:
#when humidity and windspeed are zero, the temp difference in apparent temp and actual temp is 2.4381
#as humidity increases by 1, then temp difference decreases by 3.0292
#as windspeed increases by 1, then temp difference decreases by 0.1193
#from t-test, p-values are all close to zero.  Therefore, the coefficients are statistical significant
#when it comes to explaining its variable's relationship to the target variable as stated above.

In [4]:
#Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. 
#Are the coefficients statistically significant? Did the signs of the estimated coefficients for 
#humidity and windspeed change? Interpret the estimated coefficients.

In [12]:
weather_df['humidity_windspeed'] = weather_df['humidity']*weather_df['windspeed']

In [13]:
X= weather_df[['humidity','windspeed', 'humidity_windspeed']]
Y= weather_df['temp diff']

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              temp diff   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Sun, 18 Aug 2019   Prob (F-statistic):               0.00
Time:                        16:06:20   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0839      0

In [15]:
#based on the p-values, the new coefficients are still statistically significant.
#when interpreting the target variable in terms of interaction between the humidity and windspeed data,
#the difference in temp decreases by 0.2971 for every increase in humidity AND windspeed.  
#humidity and windspeed variables alone are still statistically significant.
#meaning humidity and windspeed on their own can still affect the target variable.
#however, the signs for humidity and windspeed coefficients changed from negative to positive. 
#that means humidity alone vs. humidity WITH windspeed affects 
#apprenttemperature and/or temperature differently.
#same is true for windspeed alone vs. windspeed WITH humidity's effect on the right side of the equation below:
#weather_df['temp diff']=weather_df['apparenttemperature'] - weather_df['temperature']